In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexSets

In [ ]:
gcs = GraphOfConvexSets()
max_trace = 10

s = gcs.add_vertex("s")
ms = s.add_variable(2)
Ss = s.add_variable((2, 2))
s.add_constraint(ms == [0, 0])
s.add_constraint(Ss == [[.1, 0], [0, 1]])

v1 = gcs.add_vertex("v1")
m1 = v1.add_variable(2)
S1 = v1.add_variable((2, 2), PSD=True)
c1 = np.array([3, 2])
v1.add_constraint(cp.norm(m1 - c1, np.inf) <= 1)
v1.add_constraint(cp.trace(S1) <= max_trace)

v2 = gcs.add_vertex("v2")
m2 = v2.add_variable(2)
S2 = v2.add_variable((2, 2), PSD=True)
c2 = np.array([6, -3])
v2.add_constraint(cp.norm(m2 - c2, 2) <= 1)
v2.add_constraint(cp.trace(S2) <= max_trace)

v3 = gcs.add_vertex("v3")
m3 = v3.add_variable(2)
S3 = v3.add_variable((2, 2), PSD=True)
c3 = np.array([7, 2])
v3.add_constraint(cp.norm(m3 - c3, 1) <= 1.5)
v3.add_constraint(cp.trace(S3) <= max_trace)

t = gcs.add_vertex("t")
mt = t.add_variable(2)
St = t.add_variable((2, 2))
t.add_constraint(mt == [10, 0])
t.add_constraint(St == [[1, 0], [0, .1]])

In [ ]:
def add_edge(tail, head):
    edge = gcs.add_edge(tail, head)
    S = edge.add_variable((4, 4), PSD=True)
    m1, S1 = tail.variables
    m2, S2 = head.variables
    edge.add_constraint(S[:2, :2] == S1)
    edge.add_constraint(S[2:, 2:] == S2)
    edge.add_cost(cp.sum_squares(m2 - m1))
    edge.add_cost(cp.trace(S1 + S2 - 2 * S[:2, 2:]))
    return edge

es1 = add_edge(s, v1)
es2 = add_edge(s, v2)
e12 = add_edge(v1, v2)
e13 = add_edge(v1, v3)
e23 = add_edge(v2, v3)
e2t = add_edge(v2, t)
e3t = add_edge(v3, t)

In [ ]:
gcs.graphviz()

In [ ]:
prob = gcs.solve_shortest_path(s, t)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

def plot_gaussian(m, S, n=100):
    p = multivariate_normal(m, S)
    X = np.linspace(m[0] - 5, m[0] + 5, n)
    Y = np.linspace(m[1] - 5, m[1] + 5, n)
    X, Y = np.meshgrid(X, Y)
    pos = np.dstack((X, Y))
    Z = p.pdf(pos)
    plt.contour(X, Y, Z)

plt.figure()
plt.gca().set_aspect('equal')
plt.axis('off')
gcs.plot_2d()

for vertex in gcs.vertices:
    if vertex.y.value is not None and vertex.y.value > .99:
        m, S = vertex.variables
        plot_gaussian(m.value, S.value)